In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
import pandas as pd

df =  pd.read_csv('../artifacts/datasets/books_final.txt')
embedding_df = pd.read_csv('../artifacts/datasets/embeddings.csv')
embds = embedding_df.values
embds.shape

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

raw_docs = TextLoader('../artifacts/datasets/tagged_descriptions.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1,chunk_overlap=0,separator="\n")
docs = text_splitter.split_documents(raw_docs)

In [ ]:
from langchain_chroma import Chroma

book_db = Chroma(
    collection_name='BooksCollection',
    embedding_function=None,
)

book_db._collection.add(
    embeddings=embds,
    documents=[doc.page_content for doc in docs],
    ids=[f"doc_{i}" for i in range(len(docs))]
)

In [ ]:
def get_similar_books(query, book_db = book_db, model = model, n_results=50):
    query_vec = model.encode([query],normalize_embeddings=True)[0]
    recommendations = book_db._collection.query(
        query_embeddings=[query_vec],
        n_results=n_results
    )
    ids = list(int(i[1:14]) for i in recommendations['documents'][0])
    return df[df['isbn13'].isin(ids)]


In [ ]:
def get_book_of_category(cat:str,df:pd.DataFrame,n_result:int)->pd.DataFrame:
    if cat!= 'all':
        return df[df['simple_categories']==cat].head(n_result)
    else:
        return df.head(n_result)
    
def get_book_with_sentimance(sentimance:str,df:pd.DataFrame)-> pd.DataFrame:
    if sentimance =='all':
        return df
    elif sentimance == 'Happy':
        return df.sort_values(by='joy',ascending=False)
    elif sentimance == 'Surprising':
        return df.sort_values(by='surprise',ascending=False)
    elif sentimance == 'Angry':
        return df.sort_values(by='anger',ascending=False)
    elif sentimance == 'Suspenseful':
        return df.sort_values(by='fear',ascending=False)
    elif sentimance == 'Sad':
        return df.sort_values(by='sadness',ascending=False)

In [ ]:
def get_recommendations(query:str,category:str,emotion:str,n_results:int=10)->pd.DataFrame:
    similar_books = get_similar_books(query)
    books_of_category = get_book_of_category(category,similar_books,n_results)
    book_with_sentimance = get_book_with_sentimance(emotion,books_of_category)
    return book_with_sentimance

In [ ]:
b1 = get_recommendations('A story book about trying to love nature','all','all',5)
b1.title.values

In [ ]:
b1.description.values